In [2]:
n = 1000
function f(x)
    A = reshape(x[1:n^2], n, n)
    B = reshape(x[n^2 + 1:2n^2], n, n)
    c = x[2n^2+1:end]
    trace(log(A * B .+ c))
end

f

In [3]:
@show N = 2n^2 + n
x = rand(N)
f(x)

  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /Users/jmxp/.julia/v0.5/IJulia/src/execute_request.jl


In [4]:
N

2001000

  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /Users/jmxp/.julia/v0.5/IJulia/src/execute_request.jl


In [5]:
include("prototype6.jl")

backprop

In [11]:
# need to make these in-place
@diff_rule *(x::Array{RDiff}, y::Array{RDiff}) begin
    incadjoint!(x, adjoint(res) * value(y)')
    incadjoint!(y, value(x)' * adjoint(res))    
end

@diff_rule +(x::Array{RDiff}, y::Array{RDiff}) begin
    incadjoint!(x, adjoint(res))
    incadjoint!(y, adjoint(res))
end

backprop

In [12]:
@time ∇f = grad(f, N)

  0.007033 seconds (7 allocations: 15.267 MB)


∇f

In [13]:
stor = similar(x)
@time ∇f(x, stor);

  1.587463 seconds (15.01 M allocations: 534.774 MB)


In [15]:
@time ∇f(x, stor);

  2.450085 seconds (15.01 M allocations: 534.631 MB, 39.47% gc time)


------------

In [7]:
# prototype 6 @ N = 180k
@time ∇f(xx, stor);

 69.803386 seconds (327.08 M allocations: 9.350 GB, 36.72% gc time)


In [9]:
# prototype6
@time ∇f(x, stor);

  1.622765 seconds (12.33 M allocations: 361.259 MB, 11.34% gc time)


In [8]:
# prototype5
@time ∇f(x, stor);

  2.969177 seconds (12.32 M allocations: 344.716 MB, 0.53% gc time)


In [7]:
# prototype4
@time ∇f(x);

  1.974257 seconds (12.33 M allocations: 361.566 MB, 9.65% gc time)


In [13]:
# prototype
@time ∇f(x);

  2.230879 seconds (23.62 M allocations: 690.825 MB, 51.35% gc time)


In [5]:
ex = quote
    @diff_rule +(x::RDiff, y::RDiff) begin
    x.adj += this.adj
    y.adj += this.adj
    end
end

macroexpand(ex)

quote  # In[5], line 2:
    begin  # /Users/jmxp/code/rdiff/prototype5.jl, line 65:
        function +(x::RDiff,y::RDiff) # /Users/jmxp/code/rdiff/prototype5.jl, line 66:
            res = map(RDiff,value(x) + value(y)) # /Users/jmxp/code/rdiff/prototype5.jl, line 67:
            n = Node(+,tuple(x,y),res) # /Users/jmxp/code/rdiff/prototype5.jl, line 68:
            push!(varstack,n) # /Users/jmxp/code/rdiff/prototype5.jl, line 69:
            res
        end # /Users/jmxp/code/rdiff/prototype5.jl, line 72:
        function forward(::typeof(+),Any[:(x::RDiff),:(y::RDiff)],res) # /Users/jmxp/code/rdiff/prototype5.jl, line 73:
            new_res = value(x) + value(y) # /Users/jmxp/code/rdiff/prototype5.jl, line 74:
            for (r,n) = zip(res,new_res) # /Users/jmxp/code/rdiff/prototype5.jl, line 75:
                r.val = n.val # /Users/jmxp/code/rdiff/prototype5.jl, line 76:
                r.adj = 0.0
            end
        end # /Users/jmxp/code/rdiff/prototype5.jl, line 80:
  

  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
  likely near /Users/jmxp/.julia/v0.5/IJulia/src/kernel.jl:31
in display_dict at /Users/jmxp/.julia/v0.5/IJulia/src/execute_request.jl


In [9]:
ex = quote
    @diff_rule *(x::Array{RDiff}, y::Array{RDiff}) begin
        x.adj += res.adj * y.val'
        y.adj += x.val' * res.adj
    end
end

macroexpand(ex)

quote  # In[9], line 2:
    begin  # /Users/jmxp/code/rdiff/prototype6.jl, line 66:
        function *(x::Array{RDiff},y::Array{RDiff}) # /Users/jmxp/code/rdiff/prototype6.jl, line 67:
            res = map(RDiff,value(x) * value(y)) # /Users/jmxp/code/rdiff/prototype6.jl, line 68:
            n = RNode(*,tuple(x,y),res) # /Users/jmxp/code/rdiff/prototype6.jl, line 69:
            push!(varstack,n) # /Users/jmxp/code/rdiff/prototype6.jl, line 70:
            res
        end # /Users/jmxp/code/rdiff/prototype6.jl, line 73:
        function backprop(::typeof(*),x::Array{RDiff},y::Array{RDiff},res) # /Users/jmxp/code/rdiff/prototype6.jl, line 74:
            begin  # In[9], line 3:
                x.adj += res.adj * (y.val)' # In[9], line 4:
                y.adj += (x.val)' * res.adj
            end # /Users/jmxp/code/rdiff/prototype6.jl, line 75:
            nothing
        end
    end
end